In [1]:
import numpy as np

# Parameters
n, p = 50, 40  # Dimensions of the matrix
noise_std = 0.1  # Standard deviation of the homoscedastic noise

# Generate two random vectors
u = np.random.rand(n)  # Vector of length n
v = np.random.rand(p)  # Vector of length p

# Create a rank-1 matrix as the outer product of u and v
rank_1_matrix = np.outer(u, v)

# Add homoscedastic Gaussian noise (constant variance across the matrix)
noise = np.random.normal(0, noise_std, size=(n, p))
noisy_matrix = rank_1_matrix + noise

# Print the original and noisy matrix
print("Rank-1 Matrix (Outer Product):")
print(rank_1_matrix)
print("\nNoisy Matrix (with Homoscedastic Noise):")
print(noisy_matrix)


Rank-1 Matrix (Outer Product):
[[0.32923638 0.75967647 0.25226516 ... 0.821511   0.49296489 0.80523334]
 [0.14119261 0.32578632 0.1081836  ... 0.35230398 0.21140739 0.34532332]
 [0.35151226 0.81107559 0.26933323 ... 0.8770938  0.52631851 0.8597148 ]
 ...
 [0.06990884 0.16130691 0.05356506 ... 0.17443662 0.10467435 0.17098028]
 [0.01986045 0.04582579 0.01521733 ... 0.04955582 0.02973701 0.04857391]
 [0.06774226 0.15630776 0.05190499 ... 0.16903057 0.10143033 0.16568135]]

Noisy Matrix (with Homoscedastic Noise):
[[ 0.33224651  0.87779205  0.19875543 ...  0.73371248  0.54667201
   0.81683282]
 [ 0.05263011  0.37345303  0.17407644 ...  0.33412836  0.28495226
   0.18484555]
 [ 0.49212496  0.79373411  0.18996489 ...  0.79034719  0.69901991
   0.81019339]
 ...
 [-0.01985821  0.21939216  0.13137711 ...  0.22127359  0.11954033
   0.11959109]
 [ 0.02632889  0.11817174 -0.08060754 ...  0.13110124 -0.03586799
   0.15482944]
 [ 0.22347839  0.05882682  0.01031703 ...  0.08775598  0.08140056
   0.07

In [2]:
import sys
import os
import matplotlib.pyplot as plt

# Add the path to utils.py
sys.path.append(r"c:\Document\Serieux\Travail\python_work\cEBNM_torch\py")
from cEBMF import *

In [3]:
mycebmf= cEBMF(data= noisy_matrix)
K=5
# Perform udv_si transformation
U, s, Vt = np.linalg.svd(noisy_matrix, full_matrices=False)


In [4]:
K = np.min([K, U.shape[1]])
print(K)

5


In [5]:
mycebmf.init_LF()
print(mycebmf.L[:,1])
print(mycebmf.F[:,1])


[-0.07584928  0.29239591  0.20289332 -0.2700468   0.00328819  0.00760711
  0.07378517  0.1620958   0.1325365   0.03483537 -0.29467611  0.17498967
 -0.03761576 -0.02555254  0.14263951  0.23491303 -0.10369515  0.02672734
 -0.12683411  0.05624148  0.00685798 -0.13760043 -0.11226447  0.03869249
 -0.10036739  0.10415984 -0.19795204 -0.42499903 -0.01898705 -0.11776222
 -0.02702603  0.06770056  0.00770695 -0.29130794  0.36521024 -0.0135707
 -0.25642969 -0.14326427 -0.12100757  0.29772406  0.13883331 -0.07803124
  0.26813903 -0.08250533 -0.0509668   0.30848567  0.23334481 -0.03207058
 -0.24682086 -0.12011527]
[-0.06842796 -0.05634831  0.15600138  0.16177327  0.14144594 -0.15167385
  0.14331333 -0.14670993  0.11278013  0.04255068  0.2177631   0.15024693
 -0.23011802 -0.14676706 -0.17681841 -0.12761541 -0.08786753 -0.22573862
 -0.12609208  0.00178929 -0.11650182 -0.21358739  0.24100298  0.37149825
  0.24101152 -0.27345426  0.0176156  -0.07512588  0.07495517  0.04018919
  0.06341382 -0.17399575 -

In [9]:
np.sum ( (Y_fit -Y_fit2 ) )

0.0

In [8]:
 
            # When K>1, sum the outer products across all K components
prod_square_firstmom = np.sum(
                [np.outer(self.loading[:, k] ** 2, self.factor[:, k] ** 2) for k in range(self.K)], axis=0
            )
            prod_sectmom = np.sum(
                [np.outer(self.loading2[:, k], self.factor2[:, k]) for k in range(self.K)], axis=0
            )

        self.update_fitted_val()  # Update fitted values Y_fit

        # Compute R2 as per the formula
        R2 = (self.Y - self.Y_fit) ** 2 - prod_square_firstmom + prod_sectmom
 

IndentationError: unexpected indent (527071822.py, line 9)